# Models test

## Loading Modules

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import modelos
from sklearn.model_selection import StratifiedKFold    
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import scipy.io
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
callbacks = [reduce]

## Loading dataset

In [2]:
window_size_=40
data = scipy.io.loadmat('UAH_timeSeries/UAH')
data_target = data.get('data_target')
clase  = data.get('clases')
acc  = data.get('acc')
gps  = data.get('gps')
proc_vehicle  = data.get('proc_vehicle')
proc_openstreetmap  = data.get('proc_openstreetmap')
proc_lane  = data.get('proc_lane')
num_clases = np.unique(clase).shape[0]

axes=3

## kfold cross validation

In [3]:
## 10 folds
folds_=10

## indexes for every fold
cv = StratifiedKFold(n_splits=folds_, shuffle=True)
train_index_= []
test_index_=[]
for train_index, test_index in cv.split(acc,clase):  
    train_index_.append(train_index)
    test_index_.append(test_index) 

## Classifier

### keras model build function

crearModelo (from modelos.py): This function make a keras model with a feature extraction, feature union, and classification stages.
             The layers to be use in each stage are define by the user. 
			 
**num_clases**   :  The number of classes in the dataset.  
**window_size**  :  Number of samples in each channel.  
**ejes**         :  Number of Channels.  
**filtros**      :  The number of filters used in each convolutional layer (inception layer included).  
**cant_entradas**:  Number of inputs (modalities) in the dataset-  
**extraccion:**  :  Feature extraction layer type to be used.  
&nbsp;&nbsp;&nbsp;Options:  
&nbsp;&nbsp;&nbsp;-  "1I"    : 1 inception layer.  
&nbsp;&nbsp;&nbsp;-  "1Conv" : 1 convolutional layer.  
&nbsp;&nbsp;&nbsp;-  "21"    : 2 inception layers.  
&nbsp;&nbsp;&nbsp;-  "2Conv" : 2 convolutional layer.  
&nbsp;&nbsp;&nbsp;-  "2Ires" : 2 inception layers with residual connection.  
**tipo_union**   :  Feature fusion layer type to be used.  
&nbsp;&nbsp;&nbsp;Options:  
&nbsp;&nbsp;&nbsp;-  mGMU   : Minimal gated multimodal unit layer.  
&nbsp;&nbsp;&nbsp;-  GMU    : Gated multimodal unit layer.  
&nbsp;&nbsp;&nbsp;-  LSTM   : Long short-term memory layer.  
&nbsp;&nbsp;&nbsp;-  Concat : Concatenation layer.  
**kernel_size_** :  Size of the kernel(filter) in convolutional layers.  

In [4]:
recall = []
precision =  []
facc = []
f1score = []
auc = []

for particion in range (10):                        
    train_index = train_index_[particion]
    test_index = test_index_[particion]
    y_train, y_test = data_target[train_index], data_target[test_index]
    
    #Getting the channels to be used for all inputs
    acc1_train = acc[train_index][:,:,[1,2,3]]
    acc2_train = acc[train_index][:,:,[4,5,6]]
    acc3_train = acc[train_index][:,:,[7,8,9]]
    acc1_test, acc2_test, acc3_test = acc[test_index][:,:,[1,2,3]], acc[test_index][:,:,[4,5,6]], acc[test_index][:,:,[7,8,9]]
    gps1_train, gps2_train = gps[train_index][:,:,[0,6,7]], gps[train_index][:,:,[1,2,3]]
    gps1_test, gps2_test = gps[test_index][:,:,[0,6,7]], gps[test_index][:,:,[1,2,3]]        
    proc_lane_train = proc_lane[train_index][:,:,[0,1,2]]
    proc_lane_test  = proc_lane[test_index][:,:,[0,1,2]]        
    proc_vehicle_train = proc_vehicle[train_index][:,:,[0,1,2]]
    proc_vehicle_test  = proc_vehicle[test_index][:,:,[0,1,2]]        
    proc_openstreetmap_train = proc_openstreetmap[train_index][:,:,[0,3,4]]
    proc_openstreetmap_test = proc_openstreetmap[test_index][:,:,[0,3,4]]  
    
    ##delete model if exist
    try:
        del model           
    except(NameError):
        print("Out of memory")
    #
    #New Model
    #  
    model = modelos.crearModelo(num_clases=5, window_size=window_size_, ejes=axes, filtros=32, cant_entradas=8, extraccion="2Ires", tipo_union="mGMU")            
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])             
    
    #train
    model.fit([acc1_train,acc2_train,acc3_train,gps1_train,gps2_train,proc_lane_train,proc_vehicle_train,proc_openstreetmap_train], y_train, epochs=250, verbose=False, callbacks=callbacks)
            
    #test
    pred = model.predict([acc1_test,acc2_test,acc3_test,gps1_test,gps2_test,proc_lane_test,proc_vehicle_test,proc_openstreetmap_test])    
    pred = np.argmax(pred, axis=1)            
    pred = to_categorical(pred, num_clases)    
    
    #Metrics
    recall_ = dict()
    precision_ =  dict()
    acc_ = dict()
    f1score_ = dict()
    fpr=dict()
    tpr=dict()
    roc_auc=dict()
    for i in range(num_clases):        
        recall_[i]    = metrics.recall_score(y_test[:,i], pred[:,i])
        precision_[i] = metrics.average_precision_score(y_test[:,i], pred[:,i])
        acc_[i]       = metrics.accuracy_score(y_test[:,i], pred[:,i])
        f1score_[i]   = metrics.f1_score(y_test[:,i], pred[:,i])
        fpr[i], tpr[i], _ = metrics.roc_curve(y_test[:,i], pred[:,i])
        roc_auc[i]=metrics.auc(fpr[i], tpr[i])
    auc.append(roc_auc)
    recall.append(recall_)
    precision.append(precision_)
    facc.append(acc_)
    f1score.append(f1score_)
    

Out of memory
4/4 [==============================] - 1s 20ms/step


## Metrics

In [5]:
f1score_promedio = np.zeros((num_clases,1))
recall_promedio = np.zeros((num_clases,1))
acc_promedio = np.zeros((num_clases,1))
precision_promedio = np.zeros((num_clases,1))
auc_promedio = np.zeros((num_clases,1))

for i in range(10):
    for j in range(num_clases):
        f1score_promedio[j] += f1score[i][j]
        recall_promedio[j] += recall[i][j]
        acc_promedio[j] += facc[i][j]
        precision_promedio[j] += precision[i][j]
        auc_promedio[j] += auc[i][j]

precision_promedio = precision_promedio/10
recall_promedio = recall_promedio/10
acc_promedio = acc_promedio/10
f1score_promedio = f1score_promedio/10
auc_promedio = auc_promedio/10

In [6]:
print("Precision: "+ str(precision_promedio))
print("Recall: "+ str(recall_promedio))
print("Accuracy: "+ str(acc_promedio))
print("F1-Score: "+ str(f1score_promedio))
print("AUC: "+ str(auc_promedio))

Precision: [[0.74773786]
 [0.88413689]
 [0.91154313]
 [0.93230539]
 [0.64654639]]
Recall: [[0.84673203]
 [0.93187831]
 [0.94368421]
 [0.94316239]
 [0.79921053]]
Accuracy: [[0.95232465]
 [0.96493726]
 [0.98203024]
 [0.98022844]
 [0.91903153]]
F1-Score: [[0.84690567]
 [0.93031902]
 [0.94808606]
 [0.95731986]
 [0.77887662]]
AUC: [[0.90949619]
 [0.95394116]
 [0.96695639]
 [0.96746355]
 [0.87219843]]
